In [1]:
user.tab_settings.shop_tab.show_shop_tab
user.follower_count

NameError: name 'user' is not defined

In [81]:
from Parallelizer import make_parallel
from common import Common

In [82]:
import requests

def request_user_info(username):
    url = f"https://tokapi-mobile-version.p.rapidapi.com/v1/user/{username}"

    headers = {
        "X-RapidAPI-Key": "e8a30fd0f4msh701c26e71077ec9p19f1b9jsn40ad6b96572a",
        "X-RapidAPI-Host": "tokapi-mobile-version.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers)
    return response.json()

In [83]:
@make_parallel
def get_user_info(username):
    if "http" in username:
        username = username.split("/")[-1]
    
    print(f"Getting user info for {username}")
    user_info = request_user_info(username)
    if message:= user_info.get("message"):
        print(f"Error: {message}")
        return {"username": username, "error": message}
    
    mapping_data ={
        "has_shop": ['user', 'tab_settings', 'shop_tab', 'show_shop_tab'],
        "num_follower": ['user', 'follower_count'],
    }
    
    transformed_data = Common.mapping_data(user_info, mapping_data)
    
    return  {"username": username} | transformed_data 

In [93]:
with open("tiktok_link.txt", "r", encoding="utf-8") as f:
    urls = f.readlines()
    
urls = [url.strip() for url in urls if len(url.strip()) > 0]

In [94]:
import time


user_infos = []

for i in range(0, len(urls), 400):
    time_start = time.time()
    user_infos_ = get_user_info(urls[i:i+400])
    print(f"Epoch {i+400}: {len(user_infos_)} users")
    time_taken = time.time() - time_start
    # time.sleep(60 - time_taken + 10)
    user_infos.extend(user_infos_)
    print(f"Total users: {len(user_infos)}")

Getting user info for @sogastorevn
Getting user info for @bebong_shop1
Getting user info for @nguyenkhafishing
Getting user info for @giadungkaymall123
Getting user info for @sennheiservn
Getting user info for @xuongdobosll
Getting user info for @maymaygiadinhriccar
Getting user info for @aquaphorvietnam?lang=en
Getting user info for @soupcaonangluongsuppro
Getting user info for @sakuzavietnam
Getting user info for @greenlife_vn
Getting user info for @yorokobi_airu_official
Getting user info for @dermaestheticsvietnam
Getting user info for @originsg_official
Getting user info for @starlord.sg
Getting user info for @limevn_official
'user'
'user'
Getting user info for @rhino.store
'shop_tab'
Getting user info for @cupnvase
'shop_tab'
Getting user info for @paynastore
'shop_tab'
Getting user info for @truonggiashop
'shop_tab'
Getting user info for @capuchashop
'shop_tab'
Getting user info for @ht_0390
'shop_tab'
Getting user info for @xuongpoxe
'shop_tab'
Getting user info for @mauvechick

In [95]:
len(user_infos)

53

In [86]:
def number2string(number):
    if number < 10000:
        return str(number)
    for char in ['K', 'M', 'B', 'T']:
        number = number / 1000
        if number < 1000:
            return f"{number:.1f}{char}"
    return f"{number:.1f}T"

def string2number(string):
    if string[-1] in ['K', 'M', 'B', 'T']:
        number = float(string[:-1])
        for i, char in enumerate(['K', 'M', 'B', 'T']):
            if string[-1] == char:
                return int(number * 1000 ** (i+1))
    return int(float(string))

In [87]:
string2number(number2string(123457)),number2string(134567)

(123500, '134.6K')

In [88]:
df[df['num_follower'].isna()]


,username,has_shop,num_follower,error,url,num_follower_1,num_follower_2


In [89]:
df.head()

,username,has_shop,num_follower,error,url,num_follower_1,num_follower_2
0,@lihkiestoresg,None,0.0,NaN,https://www.tiktok.com/@lihkiestoresg,0.0,0
1,@beetech.store,None,931.0,NaN,https://www.tiktok.com/@beetech.store,931.0,931
2,@tocxukorea,True,327.0,NaN,https://www.tiktok.com/@tocxukorea,327.0,327
3,@pikapi_store,True,24448.0,NaN,https://www.tiktok.com/@pikapi_store,24.4K,24400
4,@mamasu.bear,True,49739.0,NaN,https://www.tiktok.com/@mamasu.bear,49.7K,49700


In [90]:
user_infos

[{'username': '@linhkienstoresg', 'has_shop': True, 'num_follower': 22564},
 {'username': '@beetech.store', 'has_shop': True, 'num_follower': 931},
 {'username': '@tocxukorea', 'has_shop': None, 'num_follower': 327},
 {'username': '@pikapi_store', 'has_shop': None, 'num_follower': 24449},
 {'username': '@mamasu.bear', 'has_shop': True, 'num_follower': 49736},
 {'username': '@hqtstore', 'has_shop': True, 'num_follower': 200},
 {'username': '@maytinhbanghaiphong', 'has_shop': True, 'num_follower': 41060},
 {'username': '@anhphi.bantao', 'has_shop': None, 'num_follower': 101196},
 {'username': '@dichvugiaminh.com', 'has_shop': None, 'num_follower': 10},
 {'username': '@boxxshopvn', 'has_shop': None, 'num_follower': 8},
 {'username': '@nhanpcbienhoa', 'has_shop': None, 'num_follower': 1002},
 {'username': '@skinavisofficial', 'has_shop': None, 'num_follower': 6},
 {'username': '@leafcosmetic', 'has_shop': True, 'num_follower': 58},
 {'username': '@haianhfamily199', 'has_shop': None, 'num_f

In [96]:
import pandas as pd

df = pd.DataFrame(user_infos)
df["url"] = df["username"].apply(lambda x: f"https://www.tiktok.com/{x}" )
df["num_follower"][df['num_follower'].isna()] = 0
df["num_follower_1"] = df["num_follower"].apply(number2string)
df["num_follower_2"] = df["num_follower_1"].apply(string2number)
df.to_csv("tiktok_user_info.csv", index=False)


C:\Users\DUY\AppData\Local\Temp\ipykernel_3096\3211172353.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["num_follower"][df['num_follower'].isna()] = 0


In [92]:
float("1.0")

1.0